In [1]:
from langchain_community.llms import Ollama 
from langchain_community.embeddings import OllamaEmbeddings

MODEL = "llama3.1:latest"

model = Ollama(model=MODEL)
embeddings = OllamaEmbeddings()

In [2]:
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader("docs/manual-lpcr.pdf")
pages = loader.load_and_split()
pages

[Document(metadata={'source': 'docs/manual-lpcr.pdf', 'page': 0}, page_content='MANUAL DEUSUARIO\nBÁSCULA ELECTRÓNICA\nREV0ANTES DE OPERAR SU BÁSCULA, FAVOR DE LEER \nESTE MANUAL CUIDADOSAMENTE.\nCODIGO: 13100930\nSeries L -PCR'),
 Document(metadata={'source': 'docs/manual-lpcr.pdf', 'page': 1}, page_content='2 Bascula electrónicaÍNDICE\nÍndice\nAdvertencias Importantes\nIntroducción\nInstrucciones de instalación\nNombre y localización de piezas    \nTeclado\nVisor/Pantalla\nDescripción de las operaciones\nCómo pesar un producto\nComo utilizar la tecla TARA\nCómo introducir precios     \nFunción PRECIO FIJO\nCómo usar la Referencia a CERO  \nSuma de productos\nVisualizar total de la venta al momento  \nVisualizar Venta Total \nBorrar Venta Total \nGrabar las memorias PLU`s\nRecordar una memoria\nCalcular Cambio \nFunción de apagado automático\nIndicador de batería baja \nUso de la batería\nOtros mensaje importantes de la bascula\nMantenimiento y limpieza\nGuía para la localización de a

In [3]:
from langchain.prompts import PromptTemplate

template = """
Responde la pregunta basado en el contexto de abajo. Si no puedes
responder la pregunta, responde "No lo se"

Context: {context}

Question: {question}
"""

prompt = PromptTemplate.from_template(template)
print(prompt.format(context="Esto es un contexto", question="Esto es una pregunta"))


Responde la pregunta basado en el contexto de abajo. Si no puedes
responder la pregunta, responde "No lo se"

Context: Esto es un contexto

Question: Esto es una pregunta



In [4]:
from langchain_community.vectorstores import DocArrayInMemorySearch

vectorstore = DocArrayInMemorySearch.from_documents(
    pages, 
    embedding=embeddings
    )

# Instantiate a retriever
retriever = vectorstore.as_retriever()

/Users/ivanlee/llama-rag-demo/.venv/lib/python3.10/site-packages/pydantic/_migration.py:283: UserWarning: `pydantic.error_wrappers:ValidationError` has been moved to `pydantic:ValidationError`.
  warnings.warn(f'`{import_path}` has been moved to `{new_location}`.')


In [5]:
#from operator import itemgetter

#chain = (
#    {
#        "context": itemgetter("question") | retriever,
#        "question": itemgetter("question")
#    } 
#    | prompt
#    | model 
#)

In [5]:
from langchain_core.runnables import RunnablePassthrough

chain = (
    {
        "context": retriever,
        "question": RunnablePassthrough()
}
| prompt
| model
)

In [7]:
questions = [
    "Como peso un producto?",
    "Como se introduce el precio por kilogramo?",
    "Como puedo borrar la venta total?",
]

for question in questions:
    print(f"Pregunta: {question}")
    print(f"Respuesta: {chain.invoke({'question': question})}")
    print()

Pregunta: Como peso un producto?
Respuesta: Según el manual, para pesar un producto, debes seguir los siguientes pasos:

1. Enciende la báscula presionando la tecla ON.
2. Espera unos segundos hasta que el visor/mantalla muestre "------".
3. Luego, coloca el producto a pesar en el platillo y vera el peso en el indicador de PESO.

Esto se explica en la página 8 del manual, en la sección "Cómo pesar un producto".

Pregunta: Como se introduce el precio por kilogramo?
Respuesta: La respuesta a la pregunta es: "Se muestra en el indicador de PRECIO"

Pregunta: Como puedo borrar la venta total?
Respuesta: Según el manual de instrucciones proporcionado, puedes borrar la venta total en la báscula electrónica pulsando la tecla correspondiente.

